Привет! Меня зовут Исмаил, я буду проверять твой проект :) Можешь обращаться ко мне на «ты». Если тебе комфортно, то и я буду к тебе так обращаться, если нет, то обязательно скажи об этом.

Пожалуйста, не удаляй мои комментарии, которые я буду оставлять в работе. Это особенно поможет, если твои проекты будут отправлены на повторную проверку. При повторных проверках у комментариев будут приписки: "Вторая итерация", "Третья итерация" и т.д. 

Ты тоже можешь реагировать на мои комментарии, но в таком случае постарайся, чтобы твои комментарии отличались от моих: например, выделяй их своим любимым цветом — так у нас не возникнет путаницы :)

Кстати, про цвета! Мои комментарии будут в следующей цветовой гамме: зелеными, желтыми и красными. Например:

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
В случае, если решение на отдельном шаге является полностью правильным. </div>

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
В случае, если решение может стать еще лучше с некоторыми корректировками </div>
 
<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
В случае, если решение какого-то шага требует значительных изменений. Проект не может быть принят с первого раза, если ревью содержит комментарии, помеченные этим цветом </div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv', sep = ',')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv', sep = ',')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b> 
    
Хвалю за использование конструкции try-except при загрузке данных

Хотя по заданию говорится, что предобработка данных не понадобится, я бы все равно поменял тип данных в столбцах 'calls' и 'messages' так как значения там похожи на целые, т. е. привёл бы их к int:

In [3]:
df[['calls', 'messages']] = df[['calls', 'messages']].astype(int)
df.dtypes

calls         int32
minutes     float64
messages      int32
mb_used     float64
is_ultra      int64
dtype: object

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Хороший первичный анализ данных

## Разбейте данные на выборки

Для начала определим саму суть задачи. Необходимо определить какой тариф будут использовать пользователи "Мегалайна" - "Смарт" или "Ультра". Значит это задача бинарной классификации. Для этой задачи целевой признак - столбец 'is_ultra' с информацией о наличии подписки "Ультра" - 1 и "Смарт" - 0:

In [4]:
df_train, df_test1 = train_test_split(df, test_size=0.4, shuffle=True, random_state=12345)
df_test, df_valid = train_test_split(df_test1, test_size=0.5, shuffle=True, random_state=12345)

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

В данном случае мы разбили набор данных на обучающую, тестовую и валидационную выборки в соотношении 60:20:20. Проверим так ли это:

In [5]:
features_train.shape

(1928, 4)

In [6]:
target_train.shape

(1928,)

In [7]:
features_valid.shape

(643, 4)

In [8]:
target_valid.shape

(643,)

In [9]:
features_test.shape

(643, 4)

In [10]:
target_test.shape

(643,)

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
Про тестовую забыл

## Исследуйте модели

### Дерево решений

Для начала исследуем дерево решений меняя гиперпапаметры связанные с глубиной дерева решений (max_deplth), миниальным количеством экземпляров, которое может содержаться в узле для его дальнейшего разбиения (min_samples_split) и миниальным количеством экземпляров, которое может содержаться в листе (min_samples_leaf):

In [11]:
max_accuracy = 0
for depth in range(1, 11):
    for samples_split in range(2, 11):
        for samples_leaf in range(1, 11):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split = samples_split, min_samples_leaf = samples_leaf) 

            model.fit(features_train, target_train)

            predictions_valid = model.predict(features_valid)
            
            accuracy = accuracy_score(target_valid, predictions_valid)

            max_accuracy = max(max_accuracy, accuracy)
            
            if accuracy == max_accuracy:
                best_depth = depth
                best_samples_split = samples_split
                best_samples_leaf = samples_leaf
    
print("max_depth =", best_depth, ',', 'min_samples_split =', best_samples_split, ',', 'min_samples_leaf =', best_samples_leaf,  ": ", max_accuracy)

max_depth = 8 , min_samples_split = 6 , min_samples_leaf = 3 :  0.8009331259720062


В данном случае параметр "accuracy" наибольший при гиперпараметрах обазначенных выше ("accuracy" примерно равен 0.8).

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Обучение верное, но рассмотрен только один гиперпараметр. Хотелось бы видеть перебор значений и у других гиперпараметров

### Случайный лес

Посмотрим на точность предсказания модели "случайный лес" меняя гиперпараметры, задающие количество деревьев и глубину дерева:

In [12]:
best_model = None
best_result = 0
for est in range(1, 16):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке при гиперпараметрах", 'n_estimators =', best_est, ',', 'max_depth =', best_depth, ':', best_result)

Accuracy наилучшей модели на валидационной выборке при гиперпараметрах n_estimators = 14 , max_depth = 9 : 0.8149300155520995


Для случайного леса параметр "accuracy" примерно 0.81 при определённых гиперпараметрах, в отличии от предыдущей модели, где было примерно 0.8. Но случайный случайный лес работает довольно медленно, поэтому я все таки предпочтение отдал бы дереву решений.

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Аналогично с прошлым комментарием, у леса можно много чего глянуть

### Логистическая регрессия

Теперь проверим "accuracy" для логистической регрессии:

In [13]:
max_accuracy = 0
sol_list = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky']
for sol in sol_list:
    model = LogisticRegression(random_state=12345, solver=sol, max_iter=100) 
    model.fit(features_train, target_train) 
    accuracy = model.score(features_valid, target_valid) 
    max_accuracy = max(max_accuracy, accuracy)
    if accuracy == max_accuracy:
        best_solver = sol
print('Лучшее значение accuracy для логистической регрессии при параметре solver =', best_solver,  ':', max_accuracy)

Лучшее значение accuracy для логистической регрессии при параметре solver = newton-cholesky : 0.7387247278382582


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Тут тоже хотелось бы видеть перебор значений параметров

Для логистической регрессии параметр "accuracy" ниже всех - 0.74 при параметре solver = 'newton-cholesky'.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Вторая итерация </b>

Теперь здорово, рассмотрено достаточное кол-во гиперпараметров. Чтобы циклы/подциклы перебора гиперпараметров не были такими громоздкими, можно испспользовать GridSearch

## Проверьте модель на тестовой выборке

Проверим модели из пункта 3 на тестовой выборке:

### Дерево решений

Проверим "дерево решений" на тестовой выборке с гиперпараметрами найденными в пункте 3.1:

In [14]:
model = DecisionTreeClassifier(random_state=12345, max_depth = 8, min_samples_split = 6, min_samples_leaf = 3) 

model.fit(features_test, target_test)

predictions_test = model.predict(features_test)
            
accuracy = accuracy_score(target_test, predictions_test)
    
print('Accuracy дерева решений на тестовой выборке', accuracy)

Accuracy дерева решений на тестовой выборке 0.8631415241057543


Значение "accuracy" тут довольно велико - 0.86. Можно с увернностью сказать, что эта модель подходит для предсказаний в данном случае.

### Случайный лес

Проверим модель на тестовой выборке:

In [15]:
model = RandomForestClassifier(random_state=12345, n_estimators = 14, max_depth = 9) 
model.fit(features_test, target_test)
result = model.score(features_test, target_test) 

print('Accuracy наилучшей модели на тестовой выборке', result)

Accuracy наилучшей модели на тестовой выборке 0.8895800933125972


Для случайного леса "accuracy" на тестовой выборке оказался ещё больше - 0.89, поэтому в данном случае я бы выбрал "случайный лес", но у данной модели есть один минус - время работы.

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Нет, мы в 3 пункте подбирали гиперпараметры по ВАЛИДАЦИОННОЙ выборке, теперь мы должны протестировать выбранные модели на тестовой выборке

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌ Вторая итерация </b> 

На тестовой выборке мы не подбираем гиперпараметры, а тестируем, проверяем выбранные модели с выбранными значениями гиперпараметров

## (бонус) Проверьте модели на адекватность

Проверим модели на адекватность, для этого посмотрим на баланс классов:

### Проверим валидационную выборку

In [16]:
zero_values_valid = target_valid.loc[target_valid == 0]
one_values_valid = target_valid.loc[target_valid == 1]
len(zero_values_valid) / (len(zero_values_valid) + len(one_values_valid))

0.6842923794712286

Все модели в 3 пункте имели качество выше, чем 0.68, поэтому они работают лучше чем модель в которой все значения нули.

### Проверим тестовую выборку

In [17]:
zero_values_test = target_test.loc[target_test == 0]
one_values_test = target_test.loc[target_test == 1]
len(zero_values_test) / (len(zero_values_test) + len(one_values_test))

0.7060653188180405

Все модели в 4 пункте имели качество выше, чем 0.71, поэтому они работают лучше чем модель в которой все значения нули.

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Можно посмотреть на баланс классов: например, если у нас 90 процентов таргета это "1", а остальные 10 - "0", то мы можем тупо выдавать каждый раз цифру 1 и получать метрику 90 %. Любая модель, которая будет выдавать качество ниже этого - плохая/неадекватная
    
dummy classifier с strategy='most_frequent' работает так же: смотрит на самый часто встречающийся класс и выдает только его (только одно значение - константу, поэтому это константная модель)

# Вывод

В данной работе нужно было выполнить следующую задачу - предсказать какую подписку лучше предложить клиентам на основании существующих данных с "accuracy" не меньше 0.75. Для выполнения данной задачи данные были разбиты на 3 выборки - обучающую, тестовую и валидационную в соотношении 60:20:20. Были протестированы модели такие как: "дерево решений", "случайный лес" и логистическая регрессия. Лучшая модель по моему мнению - "случайный лес", так как для неё параметр "accuracy" примерно равен 0.81 на валидационной выборке, что чуть меньше чем у "дерева решений" (0.8), а для тестовой выборки этот параметр для "случайного леса" ещё больше (0.89 > 0.86), единственный минус - время работы, но в данном случае для нас это не определяющий фактор, так как выборка не слишком большая. Логистическая регрессия показывает результат значительно хуже, чем остальные модели.

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b>

Хороший проект, тобою проделана большая работа, ты молодец!

Сперва похвалю за наличие везде промежуточных выводов, комментариев. Хороший код, показываешь навыки владения sklearn. Все требуемые пункты в задании в целом выполнены. Оставил пару желтых комментариев, думаю, это сможет улучшить твой проект. Также можно добавить визуализацию в проект: посмотреть на графиках, как влияет перебор параметров на результат нашей метрики

В этом проекте большое внимание уделяется подбору гиперпараметров, но это можно автоматизировать! Советую почитать на досуге про GridsearchCV и Optuna, будет полезно
    
Также отправляю ссылки на документации некоторых моделей для классификации: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    
Что обязательно стоит доработать: посмотреть побольше гиперпараметров у решающего дерева и случайного леса; повторить предназначение выборок, исправить ошибки, связанные с ними
    
Из курсов и дополнительной информации советую Yandex ML Handbook (https://academy.yandex.ru/handbook/ml) и курсы Евгения Соколова на ФКН (Машинное обучение - 1 с математикой https://github.com/esokolov/ml-course-hse и Основы машинного обучения без глубокой математики https://github.com/hse-ds/iad-intro-ds) - там можешь получить дополнительную информацию, более глубокую, возможно. http://wiki.cs.hse.ru/Заглавная_страница - тут можешь глянуть записи семинаров и лекций и другие материалы соответствующих курсов. 
    
Буду ждать доработок, удачи!

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌ Вторая итерация </b> 

Ошибка при тестировании

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Третья итерация </b>

Критические ошибки исправлены, проект стал еще лучше, принимаю. Успехов!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
